[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scksh/4ch-YOLOv8/blob/main/4ch-YOLOv8.ipynb)

# Install

In [1]:
!git clone https://github.com/scksh/4ch-YOLOv8

'4ch-YOLOv8'에 복제합니다...
remote: Enumerating objects: 956, done.
remote: Counting objects: 100% (956/956), done.
remote: Compressing objects: 100% (709/709), done.
remote: Total 956 (delta 244), reused 946 (delta 236), pack-reused 0 (from 0)
오브젝트를 받는 중: 100% (956/956), 1.95 MiB | 10.64 MiB/s, 완료.
델타를 알아내는 중: 100% (244/244), 완료.


In [2]:
import os

os.chdir("4ch-YOLOv8")

In [3]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] 그런 파일이나 디렉터리가 없습니다: 'requirements.txt'


# Dataset

In [4]:
import zipfile

from huggingface_hub import hf_hub_download

# Download dataset
zip_path = hf_hub_download(repo_id="SUMMERZETT/RGBIR", filename="RGBIR.zip", repo_type="dataset")

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall("./datasets")

RGBIR.zip:   0%|          | 0.00/7.63G [00:00<?, ?B/s]

# Initial Weight

In [7]:
from huggingface_hub import hf_hub_download

# Download pretrained model
model_path = hf_hub_download(
    repo_id="SUMMERZETT/YOLOv8_pretrained",
    filename="yolov8x_4ch_pretrained.pt",
    repo_type="model",
    local_dir="./model",
    local_dir_use_symlinks=False,
)

yolov8x_4ch_pretrained.pt:   0%|          | 0.00/137M [00:00<?, ?B/s]

# Pre-trained model

In [5]:
from huggingface_hub import hf_hub_download

# Download pretrained model
model_path = hf_hub_download(
    repo_id="SUMMERZETT/4ch-YOLOv8",
    filename="4ch-YOLOv8.pt",
    repo_type="model",
    local_dir="./pretrained",
    local_dir_use_symlinks=False,
)

/home/ricky/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


4ch-YOLOv8.pt:   0%|          | 0.00/137M [00:00<?, ?B/s]

# Training

In [8]:
!yolo task=detect train model=model/yolov8x_4ch_pretrained.pt data=ultralytics/cfg/datasets/RGBIR.yaml workers=2 epochs=100 batch=8 cos_lr=True

New https://pypi.org/project/ultralytics/8.3.170 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.154 🚀 Python-3.12.7 torch-2.2.2+cu118 CPU (Intel Core(TM) i9-10900 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=ultralytics/cfg/datasets/RGBIR.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=model/yolov8x_4ch_pretrained.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, n

# Validation

In [9]:
!python validate.py --weights runs/detect/train_result_250625/weights/best.pt --data ultralytics/cfg/datasets/RGBIR_valid.yaml --hyp runs/detect/train_result_250625/args.yaml --imgsz 640 --batch 1 --workers 0 --name valid_result

Traceback (most recent call last):
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/validate.py", line 225, in <module>
    run(**vars(opt))
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/validate.py", line 121, in run
    model = YOLO(weights)
            ^^^^^^^^^^^^^
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/ultralytics/models/yolo/model.py", line 79, in __init__
    super().__init__(model=model, task=task, verbose=verbose)
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/ultralytics/engine/model.py", line 151, in __init__
    self._load(model, task=task)
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/ultralytics/engine/model.py", line 295, in _load
    self.model, self.ckpt = attempt_load_one_weight(weights)
                            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ricky/.local/share/Trash/files/4ch-YOLOv8/ultralytics/nn/tasks.py", line 1540, in attempt_load_one_weight
    ckpt, weight = torch_safe_load(weight)  # load 

# Prediction

In [ ]:
!python predict.py --weights pretrained/4ch-YOLOv8.pt --source_rgb datasets/images/rgb_val --source_ir datasets/images/thermal --project runs/predict --name 4ch-YOLOv8_pred --imgsz 640 --conf 0.4 --iou 0.5 --device cuda